In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Load and prepare data

In [32]:
def load_data():
    '''
    A function for loading csv data into dataframe df.
    '''

    #Location of csv file
    csv_file = '../raw_data/air_pollution_data.csv'

    #Loading csv file into df dataframe
    df = pd.read_csv(csv_file)

    return df

def clean_data(df):
    '''
    A function to clean raw data:
    - Dropping unuseful columns
    - Dropping rows with year = NA
    - Dropping rows where pm10_concentration AND pm25_concentration AND no2_concentration are NA
    '''

    #Dropping columns: web_link, reference, iso3, who_ms, population_source, version, pm10_tempcov, pm25_tempcov, no2_tempcov
    df.drop(columns=['web_link',
                     'reference',
                     'iso3',
                     'who_ms',
                     'population_source',
                     'version',
                     'pm10_tempcov',
                     'pm25_tempcov',
                     'no2_tempcov'],
            inplace=True)

    #Dropping rows where year is NA (3 rows for India)
    df.dropna(subset=['year'], inplace=True)

    #Dropping rows where pm10_concentration AND pm25_concentration AND no2_concentration are NA
    df.dropna(how='all', subset=['pm10_concentration', 'pm25_concentration', 'no2_concentration'], inplace=True)

    return df

In [3]:
data = load_data()

In [4]:
data.head()

,who_region,iso3,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,4_Eur,ESP,Spain,A Coruna/ESP,2013.0,"V4.0 (2018), V4.0 (2018), V4.0 (2018), V4.0 (2...",23.238,11.491,28.841,87.0,46.0,93.0,"Urban, Urban, Suburban",NaN,NaN,246146.0,"manual, manual, manual, manual",43.367900,-8.418571,1
1,4_Eur,ESP,Spain,A Coruna/ESP,2014.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023)",27.476,15.878,19.575,96.0,88.0,95.0,"Urban, Urban, Suburban",NaN,NaN,247604.0,NaN,43.368033,-8.418233,1
2,4_Eur,ESP,Spain,A Coruna/ESP,2015.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",25.515,14.004,22.731,98.0,71.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
3,4_Eur,ESP,Spain,A Coruna/ESP,2016.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",23.057,13.160,20.204,98.0,98.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1
4,4_Eur,ESP,Spain,A Coruna/ESP,2017.0,"V6.0 (2023), V6.0 (2023), V6.0 (2023), V6.0...",26.849,14.114,21.543,97.0,97.0,98.0,"Urban, Urban, Suburban, Suburban",NaN,NaN,247604.0,NaN,43.370375,-8.422900,1


In [5]:
df = load_data()
df = clean_data(df)

In [6]:
def classify_concentrations(df):
    '''
    A function that classifies the concentrations of NO2, PM10, and PM2.5 into categories based on the European Air Quality Index (AQI) classification.
    '''
    # Define classification limits
    no2_limits = [0, 40, 90, 120, 230, 340, 1000]
    pm10_limits = [0, 10, 20, 25, 50, 75, 800]
    pm25_limits = [0, 20, 40, 50, 100, 150, 1200]

    # Classify PM10 concentrations
    df['pm10_class'] = pd.cut(df['pm10_concentration'], bins=pm10_limits, labels=[1, 2, 3, 4, 5, 6])

    # Classify PM2.5 concentrations
    df['pm25_class'] = pd.cut(df['pm25_concentration'], bins=pm25_limits, labels=[1, 2, 3, 4, 5, 6])

    # Classify NO2 concentrations
    df['no2_class'] = pd.cut(df['no2_concentration'], bins=no2_limits, labels=[1, 2, 3, 4, 5, 6])

    # Determine the target class as the maximum of the three pollutant classes
    df['target_class'] = df[['no2_class', 'pm10_class', 'pm25_class']].max(axis=1)

    # Drop the class concentration columns
    df = df.drop(columns=['pm10_class', 'no2_class', 'pm25_class'])
    return df


In [7]:
df = classify_concentrations(df)

In [8]:
df.head()

,who_region,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population,latitude,longitude,target_class
0,4_Eur,Spain,A Coruna/ESP,2013.0,23.238,11.491,28.841,"Urban, Urban, Suburban",246146.0,43.367900,-8.418571,3
1,4_Eur,Spain,A Coruna/ESP,2014.0,27.476,15.878,19.575,"Urban, Urban, Suburban",247604.0,43.368033,-8.418233,4
2,4_Eur,Spain,A Coruna/ESP,2015.0,25.515,14.004,22.731,"Urban, Urban, Suburban, Suburban",247604.0,43.370375,-8.422900,4
3,4_Eur,Spain,A Coruna/ESP,2016.0,23.057,13.160,20.204,"Urban, Urban, Suburban, Suburban",247604.0,43.370375,-8.422900,3
4,4_Eur,Spain,A Coruna/ESP,2017.0,26.849,14.114,21.543,"Urban, Urban, Suburban, Suburban",247604.0,43.370375,-8.422900,4


In [9]:
# Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = data.isna().sum()

# Percentage of missing values per column
missing_percentage = (data.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = data.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = data.duplicated().sum()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")
print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(data)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region                      0                    0.00             7
iso3                            0                    0.00           123
country_name                    0                    0.00           124
city                            0                    0.00          7182
year                            3                    0.01            13
version                         0                    0.00           380
pm10_concentration          11426                   28.50         18052
pm25_concentration          18368                   45.81         12647
no2_concentration           13164                   32.83         17265
pm10_tempcov                17695                   44.13           101
pm25_tempcov                23508                   58.63           101
no2_tempcov                 16696                   41.64           101
type_of_stations            16767   

40098

# Population API

In [10]:
import requests

In [34]:
def fetch_population(country, city):
    try:
        # Clean up the city name, removing country code if present
        city_name = city.split('/')[0].strip()
        # Construct the Open Data Soft API URL
        url = f"https://public.opendatasoft.com/api/records/1.0/search/?dataset=geonames-all-cities-with-a-population-500&q={city_name}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'records' in data and data['records']:
                # Extract population from the first result
                population = data['records'][0]['fields'].get('population', None)
                return int(population) if population else None
            else:
                print(f"No records found for {city}")
        else:
            print(f"Request failed for {city}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error fetching data for {city}: {e}")
    return None

# Clean up the 'city' column
#df['city'] = df['city'].apply(lambda x: x.split('/')[0].strip())

# Identify rows with NaN population
#rows_with_nan = df[df['population'].isna()]

# Fetch population data only for rows with NaN population
##for index, row in rows_with_nan.iterrows():
#    country = row['country_name']
#    city = row['city']
#    population = fetch_population(country, city)
#    if population:
#        df.at[index, 'population'] = population
# #       print(f"Filled missing population for {city}, {country} with {population}")
# #   else:
#        print(f"Could not find population for {city}, {country}")

#print(df)

In [12]:
data['city'] = data['city'].apply(lambda x: x.split('/')[0].strip())

In [13]:
merged_df = data.merge(df[['city', 'country_name','year', 'population']],
                       on=['city', 'country_name','year',],
                       how='left',
                       suffixes=('', '_new'))

# Update the 'population' column in 'data' with the 'population_new' values from 'df' where available
merged_df['population'] = merged_df['population_new'].combine_first(merged_df['population'])

# Drop the 'population_new' column after updating
merged_df.drop(columns=['population_new'], inplace=True)


In [14]:
#Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = merged_df.isna().sum()

# Percentage of missing values per column
missing_percentage = (merged_df.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = merged_df.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = merged_df.duplicated().sum()


# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")

print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(merged_df)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region                      0                    0.00             7
iso3                            0                    0.00           123
country_name                    0                    0.00           124
city                            0                    0.00          7136
year                            3                    0.01            13
version                         0                    0.00           380
pm10_concentration          11426                   28.50         18052
pm25_concentration          18368                   45.81         12647
no2_concentration           13164                   32.83         17265
pm10_tempcov                17695                   44.13           101
pm25_tempcov                23508                   58.63           101
no2_tempcov                 16696                   41.64           101
type_of_stations            16767   

40098

In [15]:
data2 = pd.read_csv('../raw_data/air_pollution_data_upd.csv')

In [16]:
#Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = data2.isna().sum()

# Percentage of missing values per column
missing_percentage = (data2.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = data2.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = data2.duplicated().sum()


# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")

print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(data2)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region                      0                    0.00             7
iso3                            0                    0.00           123
country_name                    0                    0.00           124
city                            0                    0.00          7136
year                            3                    0.01            13
version                         0                    0.00           380
pm10_concentration          11426                   28.50         18052
pm25_concentration          18368                   45.81         12647
no2_concentration           13164                   32.83         17265
pm10_tempcov                17695                   44.13           101
pm25_tempcov                23508                   58.63           101
no2_tempcov                 16696                   41.64           101
type_of_stations            16767   

40098

# Polution Api

In [17]:
temp = data2[['pm10_concentration','pm25_concentration','no2_concentration','year']]
nan_counts = temp.groupby('year').apply(lambda x: x.isnull().sum())

print("NaN counts in concentrations by year:")
print(nan_counts)

NaN counts in concentrations by year:
        pm10_concentration  pm25_concentration  no2_concentration  year
year                                                                   
2010.0                 678                1948                707     0
2011.0                 215                 448                500     0
2012.0                 230                 555                644     0
2013.0                 869                2277               1138     0
2014.0                 756                1834               1159     0
2015.0                1079                1858               1472     0
2016.0                1165                1727               1250     0
2017.0                1159                1654               1154     0
2018.0                1879                1692               1810     0
2019.0                1691                1633               1690     0
2020.0                1118                1580               1008     0
2021.0                 562

In [18]:
def classify_concentrations(df):
    '''
    Classifies the concentrations of NO2, PM10, and PM2.5 into categories based on the European Air Quality Index (AQI) classification.
    Sets the target class as the maximum of the three classified pollutant concentrations.
    '''
    # Define classification limits
    no2_limits = [0, 40, 90, 120, 230, 340, 1000]
    pm10_limits = [0, 10, 20, 25, 50, 75, 800]
    pm25_limits = [0, 20, 40, 50, 100, 150, 1200]

    # Classify PM10 concentrations
    df['pm10_class'] = pd.cut(df['pm10_concentration'], bins=pm10_limits, labels=[1, 2, 3, 4, 5, 6])

    # Classify PM2.5 concentrations
    df['pm25_class'] = pd.cut(df['pm25_concentration'], bins=pm25_limits, labels=[1, 2, 3, 4, 5, 6])

    # Classify NO2 concentrations
    df['no2_class'] = pd.cut(df['no2_concentration'], bins=no2_limits, labels=[1, 2, 3, 4, 5, 6])

    # Determine the target class as the maximum of the three pollutant classes
    df['target_class'] = df[['pm10_class', 'pm25_class', 'no2_class']].apply(lambda row: row.max(), axis=1)

    # Drop the intermediate class columns
    df = df.drop(columns=['pm10_class', 'no2_class', 'pm25_class'])

    return df

In [19]:
data2 = clean_data(data2)

In [20]:
#Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = data2.isna().sum()

# Percentage of missing values per column
missing_percentage = (data2.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = data2.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = data2.duplicated().sum()


# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")

print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(data2)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region                      0                    0.00             7
country_name                    0                    0.00           124
city                            0                    0.00          7136
year                            0                    0.00            13
pm10_concentration          11330                   28.32         18052
pm25_concentration          18273                   45.68         12647
no2_concentration           13068                   32.67         17265
type_of_stations            16695                   41.74           325
population                   1383                    3.46          8447
latitude                        0                    0.00         14010
longitude                       0                    0.00         13971

Number of duplicate rows in the dataset: 0


40002

In [21]:
data_complete = classify_concentrations(data2)
data_complete

,who_region,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population,latitude,longitude,target_class
0,4_Eur,Spain,A Coruna,2013.0,23.238,11.491,28.841,"Urban, Urban, Suburban",246056.0,43.367900,-8.418571,3
1,4_Eur,Spain,A Coruna,2014.0,27.476,15.878,19.575,"Urban, Urban, Suburban",246056.0,43.368033,-8.418233,4
2,4_Eur,Spain,A Coruna,2015.0,25.515,14.004,22.731,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,4
3,4_Eur,Spain,A Coruna,2016.0,23.057,13.160,20.204,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,3
4,4_Eur,Spain,A Coruna,2017.0,26.849,14.114,21.543,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,4
...,...,...,...,...,...,...,...,...,...,...,...,...
40093,6_Wpr,Republic of Korea,경기도,2017.0,57.335,36.457,0.029,NaN,NaN,37.337200,126.724100,5
40094,6_Wpr,Republic of Korea,경기도,2018.0,50.838,31.586,0.027,NaN,NaN,37.337200,126.724100,5
40095,6_Wpr,Republic of Korea,경기도,2019.0,55.568,31.013,0.028,NaN,NaN,37.337200,126.724100,5
40096,6_Wpr,China,虎英公园北,2018.0,NaN,30.649,NaN,NaN,NaN,23.012778,113.794444,2


In [22]:
#Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = data_complete.isna().sum()

# Percentage of missing values per column
missing_percentage = (data_complete.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = data_complete.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = data_complete.duplicated().sum()


# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")

print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(data_complete)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region                      0                    0.00             7
country_name                    0                    0.00           124
city                            0                    0.00          7136
year                            0                    0.00            13
pm10_concentration          11330                   28.32         18052
pm25_concentration          18273                   45.68         12647
no2_concentration           13068                   32.67         17265
type_of_stations            16695                   41.74           325
population                   1383                    3.46          8447
latitude                        0                    0.00         14010
longitude                       0                    0.00         13971
target_class                    0                    0.00             6

Number of duplicate rows in the dat

40002

###  Stations

In [23]:
def simplify_stations(station_type):
    '''
    Simplifies the station type string by removing duplicates and sorting.

    Args:
    - station_type (str): A string containing station types separated by ', ' e.g. Urban, urban, urban.

    Returns:
    - str: Simplified station types joined into a single string e.g "Urban, urban, urban" returns "Urban"

    If station_type is NaN (missing), returns 'unknown'.'''

    if pd.isna(station_type):
        return "unknown"
    unique_types = sorted(set(station_type.split(', ')))
    return ', '.join(unique_types)

def simplified_station_type(df):
    '''
    Adds a new column 'simplified_station_type' to the DataFrame 'df' based on simplifying 'type_of_stations'.

    Args:
    - df (pandas.DataFrame): The DataFrame containing the column 'type_of_stations' to be simplified.

    Returns:
    - pandas.DataFrame: The input DataFrame 'df' with an additional column 'simplified_station_type'.

    This function applies the 'simplify_stations' function to each value in the 'type_of_stations' column
    and stores the simplified result in a new column 'simplified_station_type'
    '''

    df['type_of_stations'] = df['type_of_stations'].astype('string') #converts type_of_stations column into a string in order to apply simplify_stations function
    df['simplified_station_type'] = df['type_of_stations'].apply(simplify_stations)
    return df

def impute_stations(df):
    '''
    Imputes the values of missing type_of_stations based on similar pollution metrics of know types of stations using KNN imputer'''

    #first simplify station names using simplified_station_type function
    simplified_station_type(df)

    # Manually map known types of stations to numerical labels from stations3 df
    type_mapping = {
        'Unknown': np.nan, #will need this to be nan for imputer to work
        'Urban': 1,
        'Rural': 2,
        'Suburban': 3,
        'Suburban, Urban': 4,
        'Rural, Urban': 5,
        'Rural, Suburban, Urban': 6,
        'Rural, Suburban': 7,
        'Background': 8,
        'Residential And Commercial Area': 9,
        'Traffic': 10,
        'Residential And Commercial Area, Urban Traffic': 11,
        'Background, Traffic': 12,
        'Industrial': 13,
        'Residential And Commercial Area, Urban Traffic': 14,
        'Industrial, Urban': 15,
        'Industrial, Rural, Urban': 16,
        'Residential': 17,
        'Fond Urbain, Traffic': 18,
        'Residential - industrial': 19
    }

    df['encoded_station_type'] = df['simplified_station_type'].map(type_mapping) # encode simpified_station_type column to feed into KNN imputer

    # Select features for imputation
    features = ['pm10_concentration', 'pm25_concentration', 'no2_concentration', 'encoded_station_type'] #features to be learned by imputer

    # Perform KNN imputation
    imputer = KNNImputer(n_neighbors=5) #instantiate imputer
    df_imputed = imputer.fit_transform(df[features]) #returns array with learned features

    # Assign imputed values back to DataFrame
    df['encoded_station_type_imputed'] = df_imputed[:, -1]  # Assuming encoded_station_type is the last column after imputation

    # Revert encoded_station_type back to original categorical values
    reverse_mapping = {v: k for k, v in type_mapping.items() if pd.notna(v)}  # Reverse mapping excluding NaNs. source >> https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping

    df['final_station_type'] = df['encoded_station_type_imputed'].round().astype(int).map(reverse_mapping).fillna(np.nan)

    return df

In [24]:
data_complete = impute_stations(data_complete)

In [25]:
data_complete

,who_region,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population,latitude,longitude,target_class,simplified_station_type,encoded_station_type,encoded_station_type_imputed,final_station_type
0,4_Eur,Spain,A Coruna,2013.0,23.238,11.491,28.841,"Urban, Urban, Suburban",246056.0,43.367900,-8.418571,3,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
1,4_Eur,Spain,A Coruna,2014.0,27.476,15.878,19.575,"Urban, Urban, Suburban",246056.0,43.368033,-8.418233,4,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
2,4_Eur,Spain,A Coruna,2015.0,25.515,14.004,22.731,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,4,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
3,4_Eur,Spain,A Coruna,2016.0,23.057,13.160,20.204,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,3,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
4,4_Eur,Spain,A Coruna,2017.0,26.849,14.114,21.543,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,4,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40093,6_Wpr,Republic of Korea,경기도,2017.0,57.335,36.457,0.029,<NA>,NaN,37.337200,126.724100,5,unknown,NaN,1.6,Rural
40094,6_Wpr,Republic of Korea,경기도,2018.0,50.838,31.586,0.027,<NA>,NaN,37.337200,126.724100,5,unknown,NaN,1.4,Urban
40095,6_Wpr,Republic of Korea,경기도,2019.0,55.568,31.013,0.028,<NA>,NaN,37.337200,126.724100,5,unknown,NaN,1.0,Urban
40096,6_Wpr,China,虎英公园北,2018.0,NaN,30.649,NaN,<NA>,NaN,23.012778,113.794444,2,unknown,NaN,1.4,Urban


# Preproccess

In [26]:
def encode_scale_data(df):
    # Reset index to ensure it's sequential and clean
    df = df.reset_index(drop=True)

    # Drop rows with missing values in critical columns
    df = df.dropna(subset=['country_name', 'city', 'year', 'population', 'target_class', 'latitude', 'longitude'])

    # Ensure 'target_class' is treated as a category
    df['target_class'] = df['target_class'].astype('category')

    # Convert 'year' to integer
    df['year'] = df['year'].astype(int)

    columns_to_drop = ['pm10_concentration', 'pm25_concentration', 'no2_concentration', 'type_of_stations',
                       'simplified_station_type', 'encoded_station_type', 'encoded_station_type_imputed']

    df = df.drop(columns_to_drop, axis=1)

    # Define the columns for encoding and scaling
    categorical_cols = ['who_region', 'country_name', 'final_station_type']
    numeric_cols = ['population', 'latitude', 'longitude']

    # Instantiate encoders and scalers
    onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
    scaler = StandardScaler()

    # Pipeline for encoding and scaling
    preprocessor = ColumnTransformer(
        transformers=[
            ('onehot', onehot_encoder, categorical_cols),
            ('scaler', scaler, numeric_cols)
        ],
        remainder='passthrough'  # Keep the year and target_class unchanged
    )

    # Apply transformations
    transformed_data = preprocessor.fit_transform(df)

    # Get the feature names after one-hot encoding
    ohe_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_cols)

    # Construct the final DataFrame
    final_columns = list(ohe_feature_names) + numeric_cols + ['city'] + ['year'] + ['target_class']
    df_transformed = pd.DataFrame(transformed_data, columns=final_columns)

    return df_transformed

In [27]:
final_data = encode_scale_data(data_complete)

In [28]:
final_data.head()

,who_region_2_Amr,who_region_3_Sear,who_region_4_Eur,who_region_5_Emr,who_region_6_Wpr,who_region_7_NonMS,country_name_Albania,country_name_Algeria,country_name_Andorra,country_name_Argentina,...,"final_station_type_Suburban, Urban",final_station_type_Traffic,final_station_type_Urban,final_station_type_nan,population,latitude,longitude,city,year,target_class
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.116974,0.198152,-0.333542,A Coruna,2013,3
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.116974,0.19816,-0.333536,A Coruna,2014,4
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.116974,0.198305,-0.333617,A Coruna,2015,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.116974,0.198305,-0.333617,A Coruna,2016,3
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.116974,0.198305,-0.333617,A Coruna,2017,4


## Model

In [29]:

from pycaret.classification import *


In [30]:
final_data = final_data.drop(columns=['city'])
# Train split the data
train_df, test_df = train_test_split(final_data, test_size=0.2, random_state=42)

# Verify the split
print(f"Training set size: {train_df.shape}")
print(f"Testing set size: {test_df.shape}")

Training set size: (30895, 152)
Testing set size: (7724, 152)


In [31]:
# Setup PyCaret with the training data

import logging
#logging.getLogger('pycaret').setLevel(logging.ERROR)

clf = setup(data=train_df, target='target_class', session_id=123, verbose=True)

# Compare models to find the best one
best_model = compare_models(exclude=['lightgbm', 'et'], sort='F1')

# Print details of the best model
print(best_model)
# Tune the model
#tuned_rf = tune_model(best_model)
#print(tuned_rf)

# Finalize the model to make predictions on the test set
final_model = finalize_model(best_model)

# Evaluate the model on the test set
predictions = predict_model(final_model, data=test_df)
print(predictions)
evaluate_model(final_model)

,Description,Value
0,Session id,123
1,Target,target_class
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5"
4,Original data shape,"(30895, 152)"
5,Transformed data shape,"(30895, 164)"
6,Transformed train set shape,"(21626, 164)"
7,Transformed test set shape,"(9269, 164)"
8,Categorical features,151
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.6690,0.7864,0.6690,0.6706,0.6695,0.5685,0.5686,0.4310
rf,Random Forest Classifier,0.6292,0.8806,0.6292,0.6217,0.6231,0.5113,0.5125,1.2820
gbc,Gradient Boosting Classifier,0.5946,0.0000,0.5946,0.5847,0.5717,0.4548,0.4642,4.0390
knn,K Neighbors Classifier,0.5636,0.8113,0.5636,0.5570,0.5585,0.4255,0.4263,0.9590
lr,Logistic Regression,0.5389,0.0000,0.5389,0.5160,0.5031,0.3766,0.3878,1.4510
lda,Linear Discriminant Analysis,0.5178,0.0000,0.5178,0.5061,0.5029,0.3571,0.3622,0.6380
ridge,Ridge Classifier,0.5229,0.0000,0.5229,0.5022,0.4808,0.3522,0.3649,0.4480
svm,SVM - Linear Kernel,0.5025,0.0000,0.5025,0.5181,0.4668,0.3286,0.3469,0.7040
ada,Ada Boost Classifier,0.4730,0.0000,0.4730,0.4653,0.4406,0.2890,0.2997,0.7810
dummy,Dummy Classifier,0.3410,0.5000,0.3410,0.1163,0.1734,0.0000,0.0000,0.4540


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, random_state=123, splitter='best')


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.6915,0.8002,0.6915,0.6921,0.6917,0.5971,0.5971


      who_region_2_Amr who_region_3_Sear who_region_4_Eur who_region_5_Emr  \
29073              0.0               0.0              1.0              0.0   
2323               0.0               0.0              1.0              0.0   
8879               0.0               1.0              0.0              0.0   
5503               0.0               0.0              1.0              0.0   
14326              1.0               0.0              0.0              0.0   
...                ...               ...              ...              ...   
13088              0.0               0.0              1.0              0.0   
35184              0.0               0.0              1.0              0.0   
16780              0.0               0.0              1.0              0.0   
19791              0.0               0.0              1.0              0.0   
37123              0.0               0.0              0.0              0.0   

      who_region_6_Wpr who_region_7_NonMS country_name_Albania 

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…